# Application

In [ ]:
_imageWidget = ImageWidget(display=False)
_crop = CropWidget(imageWidget=_imageWidget)
_videoSources = LiveVideoSourcesWidget(child=_crop, filename = '../videos.json')
_resize = resizeWidget = ResizeWidget(action=ResizeAction(),parent=_crop,imageWidget=_imageWidget)
_widgets = [_videoSources,_crop,_resize]
_pipeline = EasyVideoPipeLine(sources=_videoSources.sources, actions=_widgets)

#_crop.displayWidget = DisplayWidget(action=_crop.videoAction, update=_crop.updateImage)
_crop.displayWidget = DisplayWidget(update=_crop.updateImage)
display(_crop.displayWidget.hbox)

_tab  = widgets.Tab(children = [_videoSources.vbox, _crop.vbox, _resize.vbox])
_tab.set_title(0, 'videos')
_tab.set_title(1, 'crop')
_tab.set_title(2, 'resize')
display(_tab)

_imageWidget.display()

def evaluateWidgetActionList(widgets, img, start=1, end=None):
    for index in range(start,end):
        img = widgets[index].evaluate(**img)
    return img

# update interessant für Widget-Parameter
# evaluate: berechnet image-Pipeline
def print_on_select(widget):
    #     get the correct Output widget based on the index of the selected tab
    tab_idx = widget['new']  
    def _doit(image,update=False):
        #_imageWidget.evaluate(_crop.displayWidget.image)
        return _imageWidget.evaluate(evaluateWidgetActionList(_widgets, {'image':_crop.displayWidget.image}, start=1, end=tab_idx+1))
    _crop.displayWidget.update = _doit
    #_doit(_widgets[0].evaluate())
    _imageWidget.evaluate(evaluateWidgetActionList(_widgets, _widgets[0].evaluate(), start=1, end=tab_idx+1))
        
_tab.observe(print_on_select, names='selected_index')

# Coral Examples

In [ ]:
import sys
sys.path.insert(0,'/home/pi/jupyter/coral/projectposenet')
sys.path

In [ ]:
!ln -s /home/pi/jupyter/coral/projectposenet/posenet_lib/  posenet_lib 
!ln -s /home/pi/jupyter/coral/projectposenet projectposenet

## PoseEngine

In [ ]:
import random
from PIL import Image
from PIL import ImageDraw

colors = [(255,0,0,255) ,(0,255,0,255) ,(0,0,255,255),(255,255,0,255) ,(0,255,255,255) ,(255,255,255,255), \
         (125,0,0,255) ,(125,255,0,255) ,(125,0,255,255),(125,255,0,255) ,(125,255,255,255) ,(125,255,255,255)]

EDGES = (
    ('nose', 'left eye'),
    ('nose', 'right eye'),
    ('nose', 'left ear'),
    ('nose', 'right ear'),
    ('left ear', 'left eye'),
    ('right ear', 'right eye'),
    ('left eye', 'right eye'),
    ('left shoulder', 'right shoulder'),
    ('left shoulder', 'left elbow'),
    ('left shoulder', 'left hip'),
    ('right shoulder', 'right elbow'),
    ('right shoulder', 'right hip'),
    ('left elbow', 'left wrist'),
    ('right elbow', 'right wrist'),
    ('left hip', 'right hip'),
    ('left hip', 'left knee'),
    ('right hip', 'right knee'),
    ('left knee', 'left ankle'),
    ('right knee', 'right ankle'),
)

def drawPoint(image,x,y,r,rgba):
    draw = ImageDraw.Draw(image)
    leftUpPoint = (x-r, y-r)
    rightDownPoint = (x+r, y+r)
    twoPointList = [leftUpPoint, rightDownPoint]
    draw.ellipse(twoPointList, fill=rgba)

def drawPosePoints(image,poses,EDGES):    
    for pose in poses:
        #if pose.score < 0.4: continue
        rgba =  random.choice(colors)     
        #print('\nPose Score: ', pose.score)
        xys = {}
        minScoreKeypoint = 0.0
        for label, keypoint in pose.keypoints.items():
            if keypoint.score < minScoreKeypoint : continue
            #print(keypoint.point)
            #drawPoint(image,keypoint.yx[1], keypoint.yx[0],2,rgba)
            drawPoint(image,keypoint.point.x,keypoint.point.y,2,rgba)
            #kp_y = int((keypoint.yx[0] ) )
            #kp_x = int((keypoint.yx[1] ) )
            kp_y = int( keypoint.point.y)
            kp_x = int( keypoint.point.x)
            
            xys[label] = (kp_x, kp_y)
            #print(' %-20s x=%-4d y=%-4d score=%.1f' %
            #      (label, keypoint.yx[1], keypoint.yx[0], keypoint.score))
        #print(xys)
    
        for a, b in EDGES:
            if a not in xys or b not in xys: continue
            ax, ay = xys[a]
            bx, by = xys[b]
            #dwg.add(dwg.line(start=(ax, ay), end=(bx, by), stroke=color, stroke_width=2))
            draw = ImageDraw.Draw(image)
            draw.line((ax,ay,bx,by), fill=128, width=3)
            #print (ax,ay,bx,b)          

In [ ]:
from pycoral.utils import edgetpu
from PIL import Image
from tflite_runtime.interpreter import load_delegate
from tflite_runtime.interpreter import Interpreter

import collections
import enum
import math
import numpy as np
import os
import platform
import sys
import time

from projectposenet.pose_engine import PoseEngine
engine = PoseEngine('projectposenet/models/mobilenet/posenet_mobilenet_v1_075_481_641_quant_decoder_edgetpu.tflite')

In [ ]:
_img = _imageWidget2.getImage()
poses, inference_time = engine.DetectPosesInImage(_img)
print('Inference time: %.f ms' % (inference_time * 1000),'recognized:', len(poses))
drawPosePoints(_img,poses,EDGES)  
_img

## Objects

In [ ]:
import time

from PIL import Image
from PIL import ImageDraw

from pycoral.adapters import common
from pycoral.adapters import detect
from pycoral.utils.dataset import read_label_file
from pycoral.utils.edgetpu import make_interpreter

labels = read_label_file('/home/pi/jupyter/coral/pycoral/test_data/coco_labels.txt')
interpreter = make_interpreter('/home/pi/jupyter/coral/pycoral/test_data/ssd_mobilenet_v2_coco_quant_postprocess_edgetpu.tflite')
interpreter.allocate_tensors()

common.input_size(interpreter)

In [ ]:
#image = Image.open('/home/pi/jupyter/coral/pycoral/test_data/grace_hopper.bmp' )
image = _app._imageWidget2.getImage()

_, scale = common.set_resized_input(
    interpreter, image.size, lambda size: image.resize(size, Image.ANTIALIAS))

_, scale = common.set_resized_input(
    interpreter, image.size, lambda size: image)


print ('scale',scale)
print('----INFERENCE TIME----')

for _ in range(1):
    start = time.perf_counter()
    interpreter.invoke()
    inference_time = time.perf_counter() - start
    objs = detect.get_objects(interpreter,0.2, scale)
    print('%.2f ms' % (inference_time * 1000))
    
def draw_objects(draw, objs, labels):
  """Draws the bounding box and label for each object."""
  for obj in objs:
    bbox = obj.bbox
    draw.rectangle([(bbox.xmin, bbox.ymin), (bbox.xmax, bbox.ymax)],
                   outline='red')
    draw.text((bbox.xmin + 10, bbox.ymin + 10),
              '%s\n%.2f' % (labels.get(obj.id, obj.id), obj.score),
              fill='red')

print('-------RESULTS--------')
if not objs:
    print('No objects detected')

for obj in objs:
    print(labels.get(obj.id, obj.id))
    print('  id:    ', obj.id)
    print('  score: ', obj.score)
    print('  bbox:  ', obj.bbox)

image = image.convert('RGB')
draw_objects(ImageDraw.Draw(image), objs, labels)    

image

# Graph

In [ ]:
import networkx as nx

graph = nx.DiGraph()
graph.add_edges_from([("root", "a"), ("a", "b"), ("a", "e"), ("b", "c"), ("b", "d"), ("d", "e")])

graph.nodes()

In [ ]:
graph = nx.DiGraph()
graph.add_edges_from([(1, 2), (2, 3), (3, 4), (4, 1)])
nx.is_directed(graph)
nx.is_directed_acyclic_graph(graph)

In [ ]:
from matplotlib import pyplot as plt
g1 = nx.DiGraph()
g1.add_edges_from([("root", "a"), ("a", "b"), ("a", "e"), ("b", "c"), ("b", "d"), ("d", "e")])
plt.tight_layout()
nx.draw_networkx(g1, arrows=True)
plt.savefig("g1.png", format="PNG")
# tell matplotlib you're done with the plot: https://stackoverflow.com/questions/741877/how-do-i-tell-matplotlib-that-i-am-done-with-a-plot
plt.clf()

In [ ]:
import pylab as p
import networkx as nx

G = nx.Graph()
G.add_edge("A","B")
G.add_edge("A","H")
G.add_edge("H","C")
G.add_edge("B","C")
G.add_edge("B","D")

nx.draw(G)
p.show()

In [ ]:
import pylab as p
import networkx as nx


class Node(object):
    nodes = []

    def __init__(self, label):
        self._label = label

    def __str__(self):
        return self._label

nodes = [Node(l) for l in ["A","B","C","C","D","H"]]
edges = [(0,1),(0,5),(5,2),(1,3),(1,4)]

G = nx.Graph()
for i,j in edges:
    G.add_edge(nodes[i], nodes[j])

nx.draw(G)
p.show()

In [ ]:
#depth-first-search post-order. 
from networkx import DiGraph
from networkx.algorithms.traversal.depth_first_search import dfs_postorder_nodes

g = DiGraph({0: [1, 2], 1: [3, 4], 2: [5, 6]})
print(*dfs_postorder_nodes(g, source=0))


In [ ]:
G = nx.path_graph(3)
list(nx.bfs_edges(G, 0))

G = nx.path_graph(3)
root = 2
edges = nx.bfs_edges(G, root)
nodes = [root] + [v for u, v in edges]
nodes

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
nodes = ['0','1','2','3','4','5']
G = nx.DiGraph(name='G')
G.add_nodes_from(nodes)
edges = [('0','1'),('2','1'),('5','4'),('1','3'),('3','4')]
G.add_edges_from(edges)

bfs = nx.bfs_predecessors(G, source='0')
bfs = dict(bfs)
nx.draw(G, font_weight='bold', with_labels=True)
plt.show()
plt.savefig('graph.png')
print(bfs)

In [ ]:
graph = {
  '5' : ['3','7'],
  '3' : ['2', '4'],
  '7' : ['8'],
  '2' : [],
  '4' : ['8'],
  '8' : []
}

visited = [] # List for visited nodes.
queue = []     #Initialize a queue

def bfs(visited, graph, node): #function for BFS
  visited.append(node)
  queue.append(node)

  while queue:          # Creating loop to visit each node
    m = queue.pop(0) 
    print (m, end = " ") 

    for neighbour in graph[m]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)

# Driver Code
print("Following is the Breadth-First Search")
bfs(visited, graph, '5')    # function calling

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.DiGraph()
G.add_edges_from(
    [('A', 'B'), ('A', 'C'), ('D', 'B'), ('E', 'C'), ('E', 'F'),
     ('B', 'H'), ('B', 'G'), ('B', 'F'), ('C', 'G')])

val_map = {'A': 1.0,
           'D': 0.5714285714285714,
           'H': 0.0}

values = [val_map.get(node, 0.25) for node in G.nodes()]

# Specify the edges you want here
red_edges = [('A', 'C'), ('E', 'C')]
edge_colours = ['black' if not edge in red_edges else 'red'
                for edge in G.edges()]
black_edges = [edge for edge in G.edges() if edge not in red_edges]

# Need to create a layout when doing
# separate calls to draw nodes and edges
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), 
                       node_color = values, node_size = 500)
nx.draw_networkx_labels(G, pos)
nx.draw_networkx_edges(G, pos, edgelist=red_edges, edge_color='r', arrows=True)
nx.draw_networkx_edges(G, pos, edgelist=black_edges, arrows=False)
plt.show()

## topology

In [ ]:
G = nx.DiGraph()
#G.add_edges_from( [(_tmp4.actions[0], _tmp4.actions[1]), (_tmp4.actions[1], _tmp4.actions[2])] )
G.add_edges_from( [(_tab._widgets[0],_tab._widgets[1]),(_tab._widgets[1],_tab._widgets[2]), (_tab._widgets[1],_tab._widgets[3]), (_tab._widgets[2],_tab._widgets[4])  ] )

nx.draw(G, font_weight='bold', with_labels=True)
plt.show()

In [ ]:
labels = {}    
hubs = [_tab._widgets[1]]
pos = nx.spring_layout(G)
for node in G.nodes():
    if node in hubs:
        #set the node name as the key and the label as its value 
        labels[node] = node
#set the argument 'with labels' to False so you have unlabeled graph
nx.draw(G, with_labels=False)
#Now only add labels to the nodes you require (the hubs in my case)
nx.draw_networkx_labels(G,labels,pos,font_size=16,font_color='r')

In [ ]:
nx.spring_layout(G)
G.nodes

In [ ]:
#_tmp4.actions
#_tab._widgets
G.edges.keys
list(nx.topological_sort(G))

In [ ]:
def topological_sort_grouping(g):
    # copy the graph
    _g = g.copy()
    res = []
    # while _g is not empty
    while _g:
        zero_indegree = [v for v, d in _g.in_degree() if d == 0]
        res.append(zero_indegree)
        _g.remove_nodes_from(zero_indegree)
    return res

topological_sort_grouping(G)

In [ ]:
def dfs_edges(G,source=None):
    """Produce edges in a depth-first-search starting at source."""
    # Based on http://www.ics.uci.edu/~eppstein/PADS/DFS.py
    # by D. Eppstein, July 2004.
    if source is None:
        # produce edges for all components
        nodes=G
    else:
        # produce edges for components with source
        nodes=[source]
    visited=set()
    for start in nodes:
        if start in visited:
            continue
        visited.add(start)
        stack = [(start,iter(sorted(G[start])))]
        while stack:
            parent,children = stack[-1]
            try:
                child = next(children)
                if child not in visited:
                    yield parent,child
                    visited.add(child)
                    stack.append((child,iter(sorted(G[child]))))
            except StopIteration:
                stack.pop()
                
dfs_edges(G)                

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
edges = [['A','B'],['B','C'],['B','D']]
G = nx.Graph()
G.add_edges_from(edges)
pos = nx.spring_layout(G)
plt.figure()    
nx.draw(G,pos,edge_color='black',width=1,linewidths=1,\
node_size=500,node_color='pink',alpha=0.9,\
labels={node:node for node in G.nodes()})
nx.draw_networkx_edge_labels(G,pos,edge_labels={('A','B'):'AB',\
('B','C'):'BC',('B','D'):'BD'},font_color='red')
plt.axis('off')
plt.show()

# ActionStream

In [ ]:
# import the necessary packages
from threading import Thread
import sys
import cv2
# import the Queue class from Python 3
from queue import Queue

class ActionStream:
    def __init__(self, queueSize=128, action=None):
        self.action = action
        # initialize the file video stream along with the boolean
        # used to indicate if the thread should be stopped or not
        self.stopped = False
        # initialize the queue used to store frames read from
        # the video file
        self.Q = Queue(maxsize=queueSize)
        
    def start(self):
        # start a thread to read frames from the file video stream
        t = Thread(target=self.update, args=(), name="FileVideoStream")
        t.daemon = True
        t.start()
        return self      
    
    def update(self):
        # keep looping infinitely
        while True:
            # if the thread indicator variable is set, stop the
            # thread
            if self.stopped:
                return
            # otherwise, ensure the queue has room in it
            if not self.Q.full():
                # read the next frame from the file
                #(grabbed, frame) = self.capture.read()
                # if the `grabbed` boolean is `False`, then we have
                # reached the end of the video file
                #frame = self.action.readImage()
                frame = self.action.evaluate()
                grabbed = True
                if not grabbed:
                    self.stop()
                    return
                # add the frame to the queue
                self.Q.put(frame)    
                
    def evaluate(self):
        # return next frame in the queue
        return self.Q.get()
    
    def more(self):
        # return True if there are still frames in the queue
        return self.Q.qsize() > 0
    
    def stop(self):
        # indicate that the thread should be stopped
        self.stopped = True

In [ ]:
fs = ActionStream(queueSize=60,action=_tmp4).start()

In [ ]:
fs.evaluate()['image']

In [ ]:
#fs.stop()
fs.start()
fs.Q.qsize()

In [ ]:
_run = widgets.Checkbox(description='loop',value=False)
display(_run)
exe2 = HaltableActionExecutor(booleanField=_run,imageWidget=ImageWidget(display=True),sleep=0,action=fs,name='test')
exe2.start()

In [ ]:
# import the necessary packages
from threading import Thread
import sys
import cv2
# import the Queue class from Python 3
from queue import Queue

class FileVideoStream0:
    def __init__(self, queueSize=128, action=None):
        # initialize the file video stream along with the boolean
        # used to indicate if the thread should be stopped or not
        #_entry = videos[name]
        #_video  = pafy.new(_entry['url'])
        #self.stream = _video.streams[index]
        #self.capture = cv2.VideoCapture(self.stream.url)
        self.action = action
        self.stopped = False
        # initialize the queue used to store frames read from
        # the video file
        self.Q = Queue(maxsize=queueSize)
        
    def start(self):
        # start a thread to read frames from the file video stream
        t = Thread(target=self.update, args=(), name="FileVideoStream")
        t.daemon = True
        t.start()
        return self      
    
    def update(self):
        # keep looping infinitely
        while True:
            # if the thread indicator variable is set, stop the
            # thread
            if self.stopped:
                return
            # otherwise, ensure the queue has room in it
            if not self.Q.full():
                # read the next frame from the file
                #(grabbed, frame) = self.capture.read()
                # if the `grabbed` boolean is `False`, then we have
                # reached the end of the video file
                #frame = self.action.readImage()
                frame = self.action.evaluate()
                grabbed = True
                if not grabbed:
                    self.stop()
                    return
                # add the frame to the queue
                self.Q.put(frame)    
                
    def evaluate(self):
        # return next frame in the queue
        return self.Q.get()
    
    def more(self):
        # return True if there are still frames in the queue
        return self.Q.qsize() > 0
    
    def stop(self):
        # indicate that the thread should be stopped
        self.stopped = True

In [ ]:
_action.skipframes=15
fs = FileVideoStream(queueSize=60,action=_action).start()

In [ ]:
frame = fs.read() 
frame
#_imagew.setImage(frame)

In [ ]:
#_action_dict_list
_caction.dict()

In [ ]:
#_action.skipframes=10

_caction = createActionOfDict(_crop.action.toNormalized().dict())

while True:      
        try:
            frame = fs.read()
            _imagew.setImage(_caction.evaluate(image=frame['image'])['image'])
        except Exception as exception:
            print('done', str(exception))

In [ ]:
_vaction = _crop.videoAction

In [ ]:
_imagew = ImageWidget()

In [ ]:
#while fs.more():
from imutils.video import FPS
fps = FPS().start()

while True:    
    frame = fs.read()
    cv2.putText(frame, "Queue Size: {}".format(fs.Q.qsize()),
        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    _img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    _img = evaluateActionDictList(_action_dict_list,_img)
    #_imagew.setImage(_img)
    fps.update()
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

In [ ]:
fs.stop()

In [ ]:
fps.stop()
#print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
#print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

In [ ]:
fs = ActionStream(queueSize=60,action=_tmp5).start()
_frame = fs.evaluate()
_frame['image']

# VideoLoopPlayer

In [ ]:
#from queue import Queue

_action=_crop.videoAction
_action.skipframes = 60
_images = _action.readImages(40)

In [ ]:
#_images[30]
#_action.skipframes


In [ ]:
#imageWidget.setImage(_loop.evaluate()['image'])

In [ ]:
class VideoLoopAction():
    def __init__(self,images=None):
        self.index  = 0
        self.step   = 1
        self.images = images
        
    def evaluate(self):
        self.index = self.index + self.step
        if self.index > len(self.images) - 1:
            self.index = 0
    
        return {'image':self.images[self.index]}  

In [ ]:
import ipywidgets as widgets
style = {'description_width': '250px'}
layout = {'width': '500px'}

#_caction = createActionOfDict(_crop.action.toNormalized().dict())
#_vaction = fs

def on_next_button_clicked1(b):  
    #newvalue = int(frame.value) + 1
    #if newvalue == len(imgs):
    #    newvalue = 0
    #frame.value =  str(newvalue)
    #imageWidget.setImage(CropAction(**para_dict).evaluate(image=imgs[newvalue]))
    #imageWidget.setImage(_action.evaluate(image=imgs[newvalue]))
    #_img = capture.getFrame()
    #_img = imgs[newvalue]
    #_img = _vaction.readImageOfStream()
    #frame1 = fs.read()
    #_img = Image.fromarray(cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB))
    #_img = evaluateActionDictList(_action_dict_list,_img)
    #imageWidget.setImage(_img)
    frame = fs.read()
    _imagew.setImage(_caction.evaluate(image=frame['image'])['image']) 

def on_next_button_clicked(b):  
    #_img = _vaction.readImageOfStream()
    
    pass
    #_caction = _crop.action.toNormalized()
    #frame = _vaction.evaluate()
    #_img = _caction.evaluate(image=frame['image'])['image']
    #imageWidget.setImage(_img)
    
def initBufferedStream():    
    name='_hEh5wF6lxE'
    _entry = videos[name]
    _video  = pafy.new(_entry['url'])
    fs.stream = _video.streams[int(stream.value)]
    fs.capture = cv2.VideoCapture(fs.stream.url)
    
def on_download_button_clicked(b):  
    global imgs
    next_button.disabled = True
    #imgs = readImagesOfStream(video.streams[int(stream.value)],int(numberframes.value),skipframes=int(skipframes.value))
    #imgs = capture.readFrames(int(numberframes.value),skipframes=int(skipframes.value))
    global _vaction
    _vaction = VideoAction(video,index=int(stream.value),skipframes=int(skipframes.value))
    
    #initBufferedStream()
    
    frame.value = '0'
    next_button.disabled = False

def on_value_change_loop(b):
    if b['new'] == True:
        exe = HaltableExecutor(loop,sleep)
        exe.start()
        pending = asyncio.all_tasks()
        print(pending)
    else:
        for task in asyncio.all_tasks(): 
            if task.get_name() == 'test': 
                task.cancel()

    
stream     = widgets.Text(description   = 'stream',value = '1', style=style, layout=layout)
numberframes = widgets.Text(description = 'number frames',value = '10', style=style, layout=layout)
skipframes = widgets.Text(description   = 'skip frames',value = '10', style=style, layout=layout)
frame = widgets.Text(description = 'frame',value = '0', style=style, layout=layout)
sleep = widgets.Text(description = 'sleep',value = '0.0', style=style, layout=layout)
loop = widgets.Checkbox(description='loop',value=False)
#loop.observe(on_value_change_loop)    
download_button = widgets.Button(description='download', disabled=False, tooltip='resize',style=style, layout=layout)
download_button.on_click(on_download_button_clicked)  
next_button = widgets.Button(description='next', disabled=False, tooltip='resize',style=style, layout=layout)
next_button.on_click(on_next_button_clicked)  
vbox = widgets.VBox([stream,numberframes,skipframes,download_button,frame,sleep,loop,next_button])
display(vbox)
imageWidget=ImageWidget()
#imageWidget.display()

In [ ]:
#imageWidget.getImage().size
#video.streams
#VideoAction.__dict__

#_vaction.skipframes = 60

#display(DisplayWidget(action=_loop,imageWidget=imageWidget).hbox)
_loop = VideoLoopAction(images=_images)
display(DisplayWidget0(action=_loop,imageWidget=imageWidget).hbox)

In [ ]:
imageWidget.setImage(_loop.evaluate())

In [ ]:
import ipywidgets as widgets
import asyncio
import time

class HaltableExecutor:

    def __init__(self, checkbox,imageWidget,sleep,action):
        self._checkbox = checkbox
        self.imageWidget = imageWidget
        self._sleep    = sleep
        self.action  = action
        
    async def my_code(self):
        # This is your user code
        while True:
            if self._checkbox.value:
                print('running')  # put your code here
                #self.function('b')
                #self.imageWidget.setImage(self.action.evaluate())
                time.sleep(float(self._sleep.value))
            await asyncio.sleep(0.1)  # use this to temporarily give up control of the event loop to allow scheduling checkbox changes
    
    def start(self):
        print('starting')
        #asyncio.current_task().foo = "Bar"
        #asyncio.current_task().name = "#{}".format(n)
        task = asyncio.ensure_future(self.my_code())
        task.set_name('test')
        #asyncio.current_task().name = 'test'
        
#c = widgets.Checkbox(description='click me',value=False)
#display(c)        


exe = HaltableExecutor(_run,None,1,_loop)
exe.start()


In [ ]:
#ImageWidget().setImage(_tmp5.evaluate())
_tmp5.actions[0].index = 5
_tmp5.actions[0].skipframes=8

In [ ]:
_tmp5.actions[2].size = (420,360)
_tmp5.actions[2].__dict__

In [ ]:
del _tmp5.actions[2]

In [ ]:
_tmp5.actions[0].__dict__
#_tmp5.actions[0].evaluate()['image']

In [ ]:
_loop.index, _loop.step
#imageWidget.display()
_run = widgets.Checkbox(description='loop',value=False)
imageWidget=ImageWidget()
display(_run)

In [ ]:
import ipywidgets as widgets
import asyncio
import time

class HaltableExecutor:

    def __init__(self, checkbox=checkbox,sleep=sleep,class=None,function=evaluate,imageWidget=imageWidget):
        self._checkbox = checkbox
        self.imageWidget = imageWidget
        self._sleep    = sleep
        self.class     = class
        self.function  = function
        
    async def my_code(self):
        # This is your user code
        while True:
            if self._checkbox.value:
                #print('running')  # put your code here
                #self.function('b')
                self.imageWidget.setImage(self.claas.function
                time.sleep(float(self._sleep.value))
            await asyncio.sleep(0.1)  # use this to temporarily give up control of the event loop to allow scheduling checkbox changes
    
    def start(self):
        print('starting')
        #asyncio.current_task().foo = "Bar"
        #asyncio.current_task().name = "#{}".format(n)
        task = asyncio.ensure_future(self.my_code())
        task.set_name('test')
        #asyncio.current_task().name = 'test'
        
#c = widgets.Checkbox(description='click me',value=False)
#display(c)        

#exe = HaltableExecutor(loop,sleep,on_next_button_clicked)
exe = HaltableExecutor(checkbox=_run,sleep=sleep,class=_loop,imageWidget=imageWidget)
exe.start()

In [ ]:
def deleteAllTasks(name):
    for task in asyncio.all_tasks(): 
                if task.get_name() == name: 
                    task.cancel()
                
pending = asyncio.all_tasks()
#next(iter(pending)).cancel()
len(pending)
#pending
#asyncio.current_task().set_name('test')
#next(iter(pending)).get_name()
#task, = [task for task in asyncio.all_tasks() if task.get_name() == 'test']
#task.get_name()
#deleteAllTasks('test')
pending
deleteAllTasks('test')

In [ ]:
asyncio.all_tasks()
#deleteAllTasks('Task-9')

# trial

In [ ]:
import requests
response = requests.get(_videoSources.video.thumb)

img_byte_arr = io.BytesIO(response.content).read()
_imagew.image_w.value = img_byte_arr

In [ ]:
_videoSources.video.thumb

# tab

In [ ]:
#!/usr/bin/env python

from Tkinter import *
import tkMessageBox
import ttk

#---------------------------------------------------------------------
class MyTab(Frame):

    def __init__(self, root, name):
        Frame.__init__(self, root)

        self.root = root
        self.name = name

        self.entry = Entry(self)
        self.entry.pack(side=TOP)

        self.entry.bind('<FocusOut>', self.alert)
        self.entry.bind('<Key>', self.printing)

        self.bind("<Button-1>", self.clickFrame)

    #-------------------------------

    def alert(self, event):
        print 'FocusOut event is working for ' + self.name + '  value: ' + self.entry.get()
        #tkMessageBox.showinfo('alert', 'FocusOut event is working for ' + self.name + '  value: ' + self.entry.get())

    #-------------------------------

    def printing(self, event):
        print event.keysym + ' for ' + self.name

    #-------------------------------

    def clickFrame(self, event):
        print "MyTab: click at (" + str(event.x) + ", " + str(event.y) + ') for ' + self.name + " (parent name: " + self.root.tab(CURRENT)['text'] + ")"


#---------------------------------------------------------------------

class Application():

    def __init__(self):

        self.tabs = {'ky':1} 

        self.root = Tk()
        self.root.minsize(300, 300)
        self.root.geometry("1000x700")

        self.notebook = ttk.Notebook(self.root, width=1000, height=650)

#       self.all_tabs = []

        self.addTab('tab1')

        self.button = Button(self.root, text='generate', command=self.start_generating).pack(side=BOTTOM)

        self.notebook.pack(side=TOP)

    #-------------------------------

    def addTab(self, name):
        tab = MyTab(self.notebook, name)
        tab.bind("<Button-3>", self.clickTab)
        self.notebook.add(tab, text="X-"+name)
#       self.all_tabs.append(tab)

    #-------------------------------

    def clickTab(self, event):
        print "Application: click at (" + str(event.x) + ", " + str(event.y) + ') for ' + event.widget.name

        #-------------------------------

    def start_generating(self):
        if self.tabs['ky'] < 4:
            self.tabs['ky'] += 1
            self.addTab('tab'+ str(self.tabs['ky'])) 

    #-------------------------------

    def run(self):
        self.root.mainloop()

#----------------------------------------------------------------------

Application().run()     

# OLD

# OLD CropWidget

In [ ]:
class CropWidget():
    def __init__(self, action=None, parent=None, imageWidget=None, image=None, vstreams=None):
        self.image         = image
        self.result        = {'image':image} 
        self.action       = action
        self.videoAction  = None
        self.parent       = None
        self.imageWidget  = imageWidget
        self.vstreams     = vstreams
        self.index        = 0
        
        cropNormalizedArea = [0.0,0.0,1.0,1.0]    
        self.regions = dict()
        self.regions['reset'] = cropNormalizedArea
    
        if image != None:
            action.size = image.size
    
        self.reset_button = widgets.Button(description='Reset', disabled=False, tooltip='reset all values',style=style)
        self.reset_button.on_click(self.on_reset_button_clicked)  

        self.crop_name = widgets.Text(description = 'name',value = '', style=style, layout=layout,disabled=False)
        self.crop_name.on_submit(self.on_value_submit_crop_name)
        self.crop_list = widgets.Dropdown(description='region',options=self.regions.keys())
        self.crop_list.observe(self.crop_list_on_change) 
        self.crop_bookmarks = widgets.HBox([self.crop_list, self.crop_name, self.reset_button])

        _max = len(vstreams)
        if _max > 0:
            _max = _max -1
        self.imageselectw = widgets.IntSlider(value=0, min=0,max=_max,step=1, description='image',layout=layout)
        self.imageselectw.observe(self.on_value_change_imageselectw, 'value')
        self.next_image_button = widgets.Button(description='Next', disabled=False, tooltip='next image',style=style)
        self.next_image_button.on_click(self.on_next_image_button_clicked)  
        image_select_hbox = widgets.HBox([self.imageselectw,self.next_image_button])
        
        #self.cropx0 = self._IntSlider(description='x0',value=0)
        #self.cropx1 = self._IntSlider(description='x1')
        #self.cropx0 = widgets.IntSlider(value=0, step=1, description='x0',layout=layout)
        #self.cropx1 = widgets.IntSlider(value=0, step=1, description='x1',layout=layout)
        #self.cropx0.observe(self.on_value_change_crop_xmin, 'value')
        #self.cropx1.observe(self.on_value_change_crop_xmax, 'value')
        
        #self.crop_hbox_x = widgets.HBox([self.cropx0, self.cropx1])
        
        self.crop_hbox_x = widgets.HBox()
        self.crop_hbox_y = widgets.HBox()
        self._addIntCropSliders()
        
        #self.crop_hbox_y = widgets.HBox([self.cropy0, self.cropy1])
        
        self.vbox   = widgets.VBox([image_select_hbox,self.crop_bookmarks,self.crop_hbox_x, self.crop_hbox_y])
        #display(self.crop_vbox)
        #self.imageWidget.display()
        
        if action != None:
            self.initMinMax(action.size,(0,0,action.size[0]-1,action.size[1]-1))
        
    def _addIntCropSliders(self):
        self.cropx0 = self._IntSlider(description='x0')
        self.cropx1 = self._IntSlider(description='x1')
        self.cropx0.observe(self.on_value_change_crop_xmin, 'value')
        self.cropx1.observe(self.on_value_change_crop_xmax, 'value')
        self.crop_hbox_x.children = (self.cropx0,self.cropx1)
        
        self.cropy0 =  self._IntSlider(description='y0')
        self.cropy1 =  self._IntSlider(description='y1')
        self.cropy0.observe(self.on_value_change_crop_ymin, 'value')
        self.cropy1.observe(self.on_value_change_crop_ymax, 'value')
        self.crop_hbox_y.children = (self.cropy0,self.cropy1)
        
    def _FloatSlider(self,description='',value=0.0,layout=layout):
        return widgets.FloatSlider(value=value,description=description, min=0, max=1.0, step=0.001, readout_format='.3f',layout=layout)
    
    def _IntSlider(self,description='',value=0,max=0,layout=layout):
        return widgets.IntSlider(value=value, max=max,step=1, description=description,layout=layout)    
        
    # INIT    
    def init(self, action=None, parent=None):
        self.videoAction = action
        self.vstreams = action.getVstreams()
        self.image    = self.vstreams[0]
        
        self.action   = CropAction(size=self.image.size,min=[0,0], max=[self.image.size[0],self.image.size[1]])
        self.initMinMax()
        self.imageselectw.max = len(self.vstreams) - 1
        self.imageWidget.setImage(self.image) 
        self.parent = parent
        self.crop_list.options=self.parent.sources.videos[self.parent.action.id]['bookmarks'].keys()
        
    # RESULT 
    def crop_image(self):
        #return
        #print(self.image.size)
        self.result = self.action.evaluate(image=self.image)
        #print(cropped_image.size)
        self.imageWidget.setImage(self.result['image'])    
        
        
    def initMinMax(self,size=None,cropArea = None):
        if self.action == None:
            return
        
        if size == None:
            size = self.action.size
            cropArea = self.action.flat()
        
        self.cropx0.max = size[0] - 1
        self.cropx1.max = size[0] - 1
        self.cropy0.max = size[1] - 1
        self.cropy1.max = size[1] - 1

        self.cropx0.value = cropArea[0]
        self.cropy0.value = cropArea[1]
        self.cropx1.value = cropArea[2]
        self.cropy1.value = cropArea[3]    
        
    def on_value_change_crop_xmin(self,change):
        #print(type(change['new']))
        self.cropx1.min  = change['new']
        self.action.min[0] = change['new']
        self.crop_image()
            
    def on_value_change_crop_xmax(self,change):
        #print(change['new'],self.action.max)
        self.cropx0.max  = change['new']
        self.action.max[0] = change['new']
        self.crop_image()

    def on_value_change_crop_ymin(self,change):
        self.cropy1.min  = change['new']
        self.action.min[1] = change['new']
        self.crop_image()

    def on_value_change_crop_ymax(self,change):
        self.cropy0.max  = change['new']
        self.action.max[1] = change['new']
        self.crop_image()        
        
    # new bookmark event
    def on_value_submit_crop_name(self,change):
        #print(change)
        self.parent.sources.videos[self.parent.action.id]['bookmarks'][change.value] = self.action.normalized()
        #self.regions[change.value] = self.action.normalized()
        if change.value not in self.crop_list.options:
            self.crop_list.options = list(self.crop_list.options) + [change.value] 
        #(cropArea,image.size)
        #print(list(self.crop_list.options),self.crop_list.value, change.value)
        self.crop_list.value = change.value
        
    def crop_list_on_change(self,change):
        if change['type'] == 'change' and change['name'] == 'value':
            #print(self.regions[change['new']])
            #initMinMax(image.size,bookmark_dict[change['new']])
            #values = fromNormalizedMinMax(bookmark_dict[change['new']],image.size)
            #initMinMax(image.size,values)
            self.crop_name.value = change['new']
            #print('init',self.action.size,self.regions[change['new']])
            #self.initMinMax(self.action.size,self.action.fromNormalized(self.regions[change['new']]))
            self.initMinMax(self.action.size,self.action.fromNormalized(self.parent.sources.videos[self.parent.action.id]['bookmarks'][change['new']]))
            #self.initMinMax()

    def on_reset_button_clicked(self,b):
        #cropArea        = [0,0,image.size[0],image.size[1]]
        self.action.min = [0,0]
        self.action.max = [self.image.size[0],self.image.size[1]]
        self.initMinMax()
        self.crop_name.value = ''
        #crop_list.value = ''

    def updateImage(self,index):
        #newimage = getVstream(vstreams=self.vstreams, index=index)   
        newimage = self.vstreams[index]   
        newsize = newimage.size
        self.action.update(newsize)
        self.image = newimage
        #self.initMinMax(self.action.size,self.action.flat())
        self.initMinMax()
        #crop_image() 

    # select image slider
    def on_value_change_imageselectw(self,change):
        #imageselectw.disabled=True
        #global index
        self.index = change['new']
        self.updateImage(self.index)
        #imageselectw.disabled=False    

    def on_next_image_button_clicked(self,b):
        #global vstreams
        _video   = self.videoAction.video
        if self.videoAction.index != self.index:
            self.videoAction = VideoAction(video=_video,index=self.index)
        _vaction = self.videoAction
        self.image = _vaction.readImage()
        
        self.initMinMax()
        self.crop_image()
        #self.imageWidget.setImage(self.image)
        
        #self.initMinMax(self.action.size,self.action.fromNormalized(self.parent.sources.videos[self.parent.id]['bookmarks'][change['new']]))
        #self.vstreams = getVstreams(video=_video, indexes=[self.index])
        #self.updateImage(self.index)

#cropWidget = CropWidget(action=CropAction(size=image.size, min=[0,0], max=list(image.size)),imageWidget=ImageWidget(), image=image, vstreams=vstreams)   
#cropWidget = CropWidget(action=CropAction(),imageWidget=ImageWidget())


In [ ]:
class DisplayWidget0():
    def __init__(self, action=None, imageWidget=None):
        # VideoAction
        self.action       = action
        self.vstreams     = None
        self.index        = 0
        self.imageWidget  = imageWidget
        self.image        = None
        
        _max = 5
        self.next_image_button = widgets.Button(description='Next', disabled=False, tooltip='next image',style=style)
        self.next_image_button.on_click(self.on_next_image_button_clicked)  
        self.skipframes = widgets.Text(description   = 'skip frames',value = '10', style=style, layout=layout)
        image_select_hbox = widgets.HBox([self.next_image_button,self.skipframes])
        self.hbox=image_select_hbox
        
        # select image slider

    def on_next_image_button_clicked(self,b):
        #global vstreams
        #_video   = self.videoAction.video
        if self.action.index != self.index:
            self.action = VideoAction(video=self.action.video,index=self.index)
        self.action.skipframes = int(self.skipframes.value)
        #self.image = self.action.readImage()
        self.image = self.action.evaluate()
        self.imageWidget.setImage(self.image)
        
    def updateImage(self):
        #newimage = getVstream(vstreams=self.vstreams, index=index)   
        self.image = self.vstreams[self.index]   
        #newsize = newimage.size
        #self.action.update(newsize)
        self.imageWidget.setImage(self.image)
        #self.initMinMax(self.action.size,self.action.flat())
        #self.initMinMax()
        #crop_image()      

# TMP

# application Work

In [ ]:
#_crop.parent.action.id
#_crop.displayWidget.action.__dict__ 
_crop.displayWidget.action.save()

In [ ]:
_crop.action.dict()

In [ ]:
def save(filename='_action.json'):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(_crop.action.dict(), f, ensure_ascii=False, indent=4)    

save()

In [ ]:
def load(filename='_action.json'):
    with open(filename, 'r') as fp:
        file = json.load(fp)  
        return file        
    
load()

In [ ]:
_crop.action.load()

In [ ]:
#_widgets[1].videoAction.__dict__
#_crop.displayWidget.image
#_imageWidget.evaluate(_crop.displayWidget.image)
#_crop.displayWidget.image
_imageWidget.evaluate(evaluateWidgetActionList(_widgets, _crop.videoAction.evaluate(), start=1, end=3))


# Work

In [ ]:

#results_w.value = str('10')
#results_w
#_videoSources.video_urls.options = list('(128,128)' : (128,128))
#_videoSources.video_urls.options = {'Hallo':'Ntt1wwgCEOQ','Hallo2':'Ntt1wwgCEOQ'}

#for _video in _videoSources.sources.videos:
#    print(video)

In [ ]:
display(DisplayWidget(action=_crop.videoAction,imageWidget=_imageWidget).hbox)

In [ ]:
#VideoAction(sources=_videoSources.sources,id=_videoSources.id).dict()


def createActionOfDict(adict):
    return eval(adict['class'])(**adict['parameters'])
#createActionOfDict(_videoSources.sources.dict()).videos

#_crop.videoAction.dict()['parameters']['index']=2
_action = _crop.videoAction.dict()
_action= createActionOfDict(_action)

In [ ]:
_crop.action.toNormalized().dict(),_resize.action.dict(),_crop.videoAction.dict()

In [ ]:
_crop.action.dict()


In [ ]:
#_action.init(sources=_videoSources.sources)
#_videoSources.sources.dict()
_action.init(sources=_action.sources)

In [ ]:
df['parameters']['index']=2

In [ ]:
df = _action.dict()
_action= createActionOfDict(df)

In [ ]:

#{ **df['parameters'], **{'index':3,'skipframes':5} }

def modify_parameters(old,new):
    #return {'class':old['class']}
    return {'class':old['class'],'parameters':{**old['parameters'],**new}}

#_action = merge_dict(_crop.videoAction.dict(), {'index':4,'skipframes':5})
#_action= createActionOfDict(_action)
#_action.init(sources=_videoSources.sources)

#_new_parameters = {'index':3,'skipframes':5, 'sources':_videoSources.sources}
_new_parameters = {'index':3,'skipframes':5}
#_action = createActionOfDict(modify_parameters(_crop.videoAction.dict(),_new_parameters))
_action = _crop.videoAction.merge(parameters=_new_parameters)

In [ ]:
{**_crop.videoAction.__dict__,**_new_parameters}


In [ ]:
CropAction(min=(0,0),max=(1,0.5),mode=1).evaluate(image=ResizeAction(size=(700,500)).evaluate(image=_action.readImage())['image'])['image']

# Pipeline

In [ ]:
_pipeline = EasyVideoPipeLine(sources=_videoSources.sources, actions=_widgets)
_pipeline.dict()
#_tmp2 = createActionOfDict(_tmp.dict())

In [ ]:
_pipeline.save(name='test')
#_pipeline.load().dict()

In [ ]:
#_pipeline.dict()
#_pipeline._filename()
_pipeline.save()

In [ ]:
#_pipeline_dict = _pipeline.dict
_pipeline_dict = _pipeline.load()

In [ ]:
_tmp4 = EasyVideoPipeLine().from_dict(_pipeline_dict)
#_tmp4.init()
_imageWidget2 = ImageWidget(display=True)
_tmp4.actions.append(_imageWidget2)

In [ ]:
_tmp4.actions[0].setIndex(5)
_tmp4.actions[0].skipframes = 60
_tmp4.evaluate()

In [ ]:
def _update(image):
    #print(image)
    _tmp4.evaluate()
display(DisplayWidget(action=_tmp4.actions[0], update=_update).hbox)

In [ ]:

#_tmp5 = EasyVideoPipeLine().from_dict(_tmp4.dict())
_tmp5 = EasyVideoPipeLine().load()

In [ ]:
_tmp5.evaluate()['image']

# Create Widgets of pipeline

In [ ]:

_tmp4.dict()['parameters']['actions'][1]
#createActionOfDict(_tmp4.dict()['parameters']['sources'])
_tmp4.dict()['parameters']['sources']

In [ ]:
_pipeline_dict = _pipeline.load()
_tmp4 = EasyVideoPipeLine().from_dict(_pipeline_dict)
_tmp4_sources = createActionOfDict(_tmp4.dict()['parameters']['sources'])
_tmp4_video = createActionOfDict(_tmp4.dict()['parameters']['actions'][0])
_tmp4_crop  = createActionOfDict(_tmp4.dict()['parameters']['actions'][1])
_tmp4_video.init(sources=_tmp4_sources)
image = _tmp4_video.evaluate()['image']

In [ ]:
_tmp4 = EasyVideoPipeLine().load()

In [ ]:
_tmp4.actions

In [ ]:
tab  = widgets.Tab(children = [_videoSources.vbox, _crop.vbox, _resize.vbox])
tab.set_title(0, 'videos')
tab.set_title(1, 'crop')
tab.set_title(2, 'resize')
display(tab)

## tab OLD

In [ ]:
_tmp4 = EasyVideoPipeLine().load('ecke.json')

_run = widgets.Checkbox(description='loop',value=False)
display(_run)

_displayWidget = DisplayWidget(action=_tmp4.actions[0])
display(_displayWidget.hbox)  

_imageWidget2 = ImageWidget(display=False)
_tab = TabWidget(action=_tmp4.actions[0], displayWidget=_displayWidget,imageWidget=_imageWidget2)
display(_tab.tab)
_imageWidget2.display()

_tab.create_widget(_tmp4.sources,imageWidget=_imageWidget2)

_parent = None
for _action in _tmp4.actions:
    _widget = _tab.create_widget(_action,imageWidget=_imageWidget2)
    if _widget != None:
        _widget.parent = _parent
    _parent = _widget
    
_tab._widgets[1].crop_list.options = _tmp4.actions[0].sources.videos[_tmp4.actions[0].id]['bookmarks'].keys()    
_tab._widgets[1].parent = _tmp4.actions[0]
_tab._widgets[1].displayWidget = _displayWidget

_tab._widgets[2].parent = _tab._widgets[1]

exe = HaltableActionExecutor(booleanField=_run,imageWidget=_imageWidget2,sleep=0,action=_tab,name='test')
#exe.start()

In [ ]:
#_tmp4.dict()
#_tab.evaluate()
_tab._widgets[1].get_result()
_tab._widgets[2].parent

In [ ]:
_widget = CropWidget(action=CropAction(min=[0.0,0.0],max=[1.0,1.0],mode=CropAction.NORMALIZED),imageWidget=_imageWidget2)
_tab.add(_widget)
_widget.parent = _tab._widgets[len(_tab._widgets)-2]

In [ ]:


#_tab._widgets[3].action.evaluate(_tab._widgets[2].result)

#_tab._widgets[3].action.evaluate


type(_tab._widgets[2].result)


def _evaluate(**kwargs):
    print(type(kwargs))
    
#_evaluate(**_tab._widgets[2].result)    

_tab._widgets[3].action.result.keys()

In [ ]:
_widget.__dict__

In [ ]:

#_tab._widgets[3].action.result

_widgets2 = []
for key, value in _widget.result.items():
    if key != 'image':
        _widgets2.append(widgets.Text(description = key, value=str(value), style=style, layout=layout2))

In [ ]:
#_widget.vbox = widgets.VBox(_widgets2)
#_tab._widgets[3].vbox = widgets.VBox(_widgets2)
_tab._widgets_vbox[3] = widgets.VBox(_widgets2)
_tab.tab.children = _tab._widgets_vbox  

In [ ]:
_tab._widgets_vbox[3].children[1].value = str(0.4)

In [ ]:
_widget.vbox.children = _widgets2

In [ ]:
def myfunc(**kwargs):
    # kwargs is a dictionary.
    for k,v in kwargs.items():
         print ("%s = %s" % (k, v))
    if 'image' in kwargs.keys():
        print('found')
    
myfunc(result=5, image=None)

In [ ]:
_tab.add(TemplateWidget())

In [ ]:
_tab._widgets[3].result

In [ ]:
_tab.create_widget(ContrastBrightnessAction().load(),imageWidget=_imageWidget2)

In [ ]:
_tab._widgets[3].contrast_w.max = 5000
_tab._widgets[3].brightness_w.max = 2.0

In [ ]:
_tmp4.actions[0].index = 5
exe.action=_tab._widgets[1].action

In [ ]:
_run = widgets.Checkbox(description='loop',value=False)
display(_run)
exe = HaltableActionExecutor(booleanField=_run,imageWidget=_imageWidget2,sleep=0,action=_tmp4,name='test')
exe.start()

In [ ]:
asyncio.all_tasks()

In [ ]:
asyncio.Task.current_task().name = "#{}".format('hallo')



In [ ]:
deleteAllTasks('test')

In [ ]:
#_imageWidget2.evaluate(_tab.evaluateWidgetActionList(_tab._widgets[0].evaluate(), start=1, end=1))
#_tab._widgets[0].evaluate()
#_tab._widgets[1].action
#_tmp4.sources
#_tab._widgets[0].__dict__
#_displayWidget.action.evaluate()['image']
#_displayWidget.evaluate()['image']
#_displayWidget.__dict__
#_tmp4.actions[0].dict()
#_displayWidget.action.dict()
#_tab.displayWidget.image
#self.crop_list.options=self.parent.sources.videos[self.id]['bookmarks'].keys()
#_tmp4.actions[0].sources.videos[_tmp4.actions[0].id]['bookmarks'].keys()

#_tab._widgets[0].sources.__dict__
#_tmp4.actions[0].dict()
#_tmp4.actions[0].__dict__
#_tmp4.actions[0]
#_displayWidget.action.__dict__
#_tmp4.actions[0].dict()
#DisplayWidget(action=_tmp4.actions[0]).action.__dict__
#_tab._widgets[2].parent = _tab._widgets[1]
_tmp4.save()

In [ ]:
self.initMinMax(self.action.size,self.action.fromNormalized(self.parent.sources.videos[self.id]['bookmarks'][change['new']]))

In [ ]:
_tmp4 = EasyVideoPipeLine().load()

#_dict = {'CropAction':'CropWidget', 'ResizeAction':'ResizeWidget','VideoAction':'LiveVideoSourcesWidget'}
_dict = {'CropAction':'CropWidget', 'ResizeAction':'ResizeWidget', 'LiveVideoSources':'LiveVideoSourcesWidget'}

_imageWidget2 = ImageWidget(display=False)

_widgets = []
_widgets_vbox = []

def add_to_widgets(_action):
    _name = _action.__class__.__name__
    if _name in _dict.keys():
        _class = eval(_dict[_name])(action=_action, imageWidget=_imageWidget2)
        _widgets.append(_class)
        _widgets_vbox.append(_class.vbox)

add_to_widgets(_tmp4.sources)        
for _action in _tmp4.actions:
    add_to_widgets(_action)
        
_tab  = widgets.Tab(children=_widgets_vbox)
for ind, _widget in enumerate(_widgets):
    _tab.set_title(ind, _widget.action.__class__.__name__)
    
display(_tab)
_imageWidget2.display()

In [ ]:
_tmp4_video = _tmp4.actions[0]
_tmp_displayWidget = DisplayWidget(action=_tmp4_video,update=_tmp4_crop_widget.updateImage)
display(_tmp4_crop_widget.displayWidget.hbox)

In [ ]:
_tmp4_crop = _tmp4.actions[1]
_tmp4_crop_widget = CropWidget(action=_tmp4_crop)
display(_tmp4_crop_widget.vbox)

_tmp4_video = _tmp4.actions[0]
_tmp4_crop_widget.displayWidget = DisplayWidget(action=_tmp4_video,update=_tmp4_crop_widget.updateImage)
display(_tmp4_crop_widget.displayWidget.hbox)

_image = _tmp4.actions[0].evaluate()
_tmp4_crop_widget.updateImage(_image)


In [ ]:
_tmp4.actions[1].evaluate(_image)['image']

In [ ]:

_tmp4.actions[1].__dict__, _tmp4_crop_widget.action.__dict__

In [ ]:
#_tmp4.dict()
_tmp4_crop.dict()
_tmp4_video.dict()
_tmp4_video.stream.dimensions
_tmp4_crop.__dict__

In [ ]:
_tmp4.actions[1].__dict__, _tmp4_crop_widget.action.__dict__

In [ ]:
_tmp4_crop_widget = CropWidget(action=_tmp4_crop, videoAction=_tmp4_video ,parent=None)
display(_tmp4_crop_widget.vbox)

_tmp4_crop_widget.displayWidget = DisplayWidget(action=_tmp4_video,update=_tmp4_crop_widget.updateImage)
display(_tmp4_crop_widget.displayWidget.hbox)



In [ ]:
_tmp4_crop_widget.action.dict()

In [ ]:
_tmp4_sources.get_keys()
_tmp4_sources.videos[_tmp4_video.id]

In [ ]:
#_tmp4_sources_widget = LiveVideoSourcesWidget(child=_tmp4_crop_widget, filename = '../videos.json', action=_tmp4_sources, id=_tmp4_video.id)
_tmp4_sources_widget = LiveVideoSourcesWidget(filename = '../videos.json', action=_tmp4_sources, id=_tmp4_video.id)
display(_tmp4_sources_widget.vbox)

In [ ]:
_tmp4_sources_widget.child = _tmp4_crop_widget

In [ ]:
_tmp4_crop_widget.displayWidget.image

In [ ]:
_tmp4.dict()

In [ ]:
_tmp4_crop.size

In [ ]:
#_tmp4_crop.evaluate(_tmp4_video.evaluate()['image'])['image']

image = _tmp4_video.evaluate()['image']
#_tmp4_crop.fromNormalized(_tmp4_crop.flat(), size=image.size)
#_tmp4_crop.flat()
image

In [ ]:
_tmp4_crop.__dict__

In [ ]:
self.initMinMax(action.size,(0,0,action.size[0]-1,action.size[1]-1))

In [ ]:
#_tmp4_crop.__dict__
#_tmp4_video.evaluate()
#_tmp4_video.__dict__
_tmp4_crop.action.__dict__

In [ ]:
#_tmp4_crop_widget.action.__dict__
#_tmp4_crop_widget.action.normalized()
#_tmp4_crop_widget.action.min
#_tmp4_crop_widget.action.crop(_tmp4_crop_widget.action._flat())
#_tmp4_crop_widget.action._flat(), _tmp4_crop_widget.action.__dict__
#_tmp4_crop.__dict__
#_tmp4_crop.flat()
#_tmp4_crop.toWorld(image.size).__dict__
#_tmp4_crop_widget.action.__dict__
_tmp4_crop_widget.result['image']
#_tmp4_crop_widget.displayWidget.image

In [ ]:
#_tmp4_sources.__dict__
_tmp4_sources_widget.video_urls.value
#_tmp4_sources_widget.id
_tmp4_video.id

In [ ]:
#_tmp4_video.__dict__
#_tmp4_crop_widget.displayWidget.on_next_image_button_clicked('')

#action.update(size)

_tmp4_crop_widget.action.__dict__
_size = _tmp4_crop_widget.displayWidget.image.size
#_tmp4_crop_widget.action.update(_size)

_tmp4_crop_widget.action.mode == _tmp4_crop_widget.action.WORLD

_tmp4_crop_widget.action.toWorld(size=_size).__dict__

In [ ]:
#_tmp4_video.evaluate()['image']
image = _tmp4_crop_widget.displayWidget.image
#_tmp4_crop_widget.updateImage(_tmp4_crop_widget.displayWidget.image)

#_tmp4_crop_widget.action.fromNormalized(_tmp4_crop_widget.action.flat(), size=image.size)
#_tmp4_crop_widget.action.__dict__, _tmp4_crop.__dict__
#_tmp4_crop_widget.action.__dict__, _tmp4_crop_widget.action.fromNormalized(_tmp4_crop_widget.action.flat(), size=image.size).__dict__
image

In [ ]:
_tmp4_crop_widget.action.__dict__, _tmp4_crop.__dict__

In [ ]:
if isinstance(_tmp.dict(), dict):
    print('dict')


#EasyVideoPipeLine().from_dict(_tmp.dict()).sources.videos

_tmp3 = EasyVideoPipeLine().from_dict(_tmp.dict())
_tmp3.actions[0].init(sources=_tmp3.sources)
_action0 = _tmp3.actions[0]
_action0.setIndex(2)
_imageWidget2 = ImageWidget(display=True)

In [ ]:
_action0.setIndex(1)

In [ ]:
_action0.skipFrames(15)
_tmp3.actions[0].evaluate()['image']
_imageWidget2.setImage(_tmp3.actions[0].evaluate())

In [ ]:
_action1 = _tmp3.actions[1]
_action2 = _tmp3.actions[2]

_action2.algorithm = Image.LANCZOS

#_action2.evaluate(_action1.evaluate(_imageWidget2.getImage())['image'])['image']

_imageWidget2.setImage(_action1.evaluate(_imageWidget2.getImage())['image'])

#_action2.dict()

In [ ]:
#_tmp3.actions[2].algorithm = Image.LANCZOS
_tmp3.actions[2].__dict__
_tmp3.actions[0].setIndex(5)
_tmp3.actions[0].skipframes = 60

In [ ]:
_tmp3.actions[0].__dict__

In [ ]:
# set result to imagewidget
_imageWidget2.evaluate(**  evaluateActionList(_tmp3.actions, {}))

In [ ]:
_actions = [ResizeAction(size=(900,500))]
evaluateActionList(_actions, {'image':_action.readImage()})['image']

In [ ]:
#_action.sources.videos[_action.id]
_action.video

In [ ]:
_videoSources.sources.videos[list(_videoSources.sources.videos.keys())[1]]

In [ ]:
_action.evaluate()['image']

In [ ]:
CropAction(min=(0,0),max=(480,640),size=(1920,1080),mode=CropAction.WORLD).dict(), CropAction(min=(0,0),max=(480,640),size=(1920,1080),mode=CropAction.WORLD).toNormalized().__dict__